In [8]:
import pandas as pd
df = pd.read_parquet('catB_train.parquet')
#print(df.head())
#print(df.columns)
#print(df.shape)
print(df.describe())

       flg_substandard  flg_is_borderline_standard  flg_is_revised_term  \
count     16978.000000                16978.000000         16978.000000   
mean          0.040287                    0.033102             0.000707   
std           0.196638                    0.178907             0.026577   
min           0.000000                    0.000000             0.000000   
25%           0.000000                    0.000000             0.000000   
50%           0.000000                    0.000000             0.000000   
75%           0.000000                    0.000000             0.000000   
max           1.000000                    1.000000             1.000000   

       flg_is_rental_flat  flg_has_health_claim  flg_has_life_claim  \
count        16978.000000          16978.000000        16978.000000   
mean             0.012546              0.048769            0.003593   
std              0.111306              0.215391            0.059835   
min              0.000000              0

In [9]:
import os
print(os.getcwd())
new_directory = 'E:/uni/datathon/catB_train.parquet.gpg'
os.chdir(new_directory)
print(os.getcwd())

E:\uni\datathon\catB_train.parquet.gpg
E:\uni\datathon\catB_train.parquet.gpg


In [10]:
numerical_columns = df.select_dtypes(include=['number']).columns

# Print the names of numerical columns
print("Numerical Columns:")
print(numerical_columns)

Numerical Columns:
Index(['flg_substandard', 'flg_is_borderline_standard', 'flg_is_revised_term',
       'flg_is_rental_flat', 'flg_has_health_claim', 'flg_has_life_claim',
       'flg_gi_claim', 'flg_is_proposal', 'flg_with_preauthorisation',
       'flg_is_returned_mail', 'is_consent_to_mail', 'is_consent_to_email',
       'is_consent_to_call', 'is_consent_to_sms', 'is_valid_dm',
       'is_valid_email', 'is_housewife_retiree', 'is_sg_pr', 'is_class_1_2',
       'is_dependent_in_at_least_1_policy', 'f_ever_declined_la', 'hh_size',
       'n_months_last_bought_products', 'flg_latest_being_lapse',
       'flg_latest_being_cancel', 'recency_lapse', 'recency_cancel',
       'tot_inforce_pols', 'tot_cancel_pols', 'f_hold_839f8a', 'f_hold_e22a6a',
       'f_hold_d0adeb', 'f_hold_c4bda5', 'f_hold_ltc', 'f_hold_507c37',
       'f_hold_gi', 'f_ever_bought_839f8a', 'f_ever_bought_e22a6a',
       'f_ever_bought_d0adeb', 'f_ever_bought_c4bda5', 'f_ever_bought_ltc',
       'f_ever_bought_507c37',